In [27]:
# !pip install PyPortfolioOpt

### MVO New year is the variant with multiple constraints such as number of assets = 3 etc


In [1]:
import numpy as np
import pandas as pd
from itertools import combinations
from scipy.optimize import minimize
import os

In [2]:
from pypfopt import EfficientFrontier, risk_models, expected_returns, objective_functions

In [3]:
portfolio_wts_dir = r'../results/backtests/portfolio_weights'
historical_data_dir = r'../data/simulations/market_scenarios_historical'



In [93]:
# Part 1: Find portfolio weights for 2020
bt_year = 2023

ret = pd.read_csv(os.path.join(historical_data_dir, str(bt_year-1)+'_annual_ret.csv'), header=None, index_col = 0 )

ret.columns = ['ret']

vol = pd.read_csv(os.path.join(historical_data_dir, str(bt_year-1)+'_annual_vol.csv'), header=None, index_col = 0 )
vol.columns = ['vol']

corr = pd.read_csv(os.path.join(historical_data_dir, str(bt_year-1)+'_annual_corr.csv'), index_col = 0 )

rfr = 0.02
covariance_matrix = np.outer(vol, vol) * corr

In [1]:
# Optimize for maximum Sharpe ratio
ef = EfficientFrontier(ret.values, covariance_matrix)
# ef.add_objective(objective_functions.L2_reg, gamma=1)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
cleaned_weights

# This code is not helpful because applying constraint of 3 assets in portfolio is difficult. Look at the next cells 



NameError: name 'EfficientFrontier' is not defined

In [95]:
pf_wt = pd.DataFrame([round(float(x),2) for x in cleaned_weights.values()])
pf_wt

,0
0,0.00
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [96]:
pf_wt.to_csv(os.path.join(portfolio_wts_dir, 'MVO', 'MVO_year_'+str(bt_year)+'.csv'), header=None, index=False)


In [97]:
pf_wt

,0
0,0.00
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [25]:
bt_year = 2023

ret = pd.read_csv(os.path.join(historical_data_dir, str(bt_year-1)+'_annual_ret.csv'), header=None, index_col = 0 )

ret.columns = ['ret']

vol = pd.read_csv(os.path.join(historical_data_dir, str(bt_year-1)+'_annual_vol.csv'), header=None, index_col = 0 )
vol.columns = ['vol']

corr = pd.read_csv(os.path.join(historical_data_dir, str(bt_year-1)+'_annual_corr.csv'), index_col = 0 )

rfr = 0.02
covariance_matrix = np.outer(vol, vol) * corr

In [35]:
import numpy as np
import pandas as pd
import cvxpy as cp

# Sample data: Expected returns, covariance matrix, risk-free rate
np.random.seed(42)
n = 25  # Number of assets
n_assets = 3  # Number of assets to include in the portfolio
mu = ret.values.reshape(-1)  # Expected returns for each asset
cov_matrix = covariance_matrix
risk_free_rate = 0.02

# Decision variables: weights and binary variables for each asset
weights = cp.Variable(n)
binary_inclusion = cp.Variable(n, boolean=True)

target_volatility = 0.14

# Objective function: Maximize Sharpe ratio
expected_return = mu @ weights
portfolio_risk = cp.quad_form(weights, cov_matrix)
sharpe_ratio = (expected_return - risk_free_rate) / cp.sqrt(portfolio_risk)

# Constraints
constraints = [
    cp.sum(weights) == 1,                  # Weights sum to 1
    weights >= 0.05 * binary_inclusion,                          # Long-only constraint
    weights >= 0,   
    weights <= binary_inclusion,           # Enforces that if binary_inclusion[i] = 0, then weights[i] = 0
    cp.sum(binary_inclusion) == n_assets,   # Exact constraint on number of assets
    cp.quad_form(weights, cov_matrix) <= target_volatility**2
]

# Set up and solve the problem
problem = cp.Problem(cp.Maximize(expected_return), constraints)
problem.solve(solver=cp.ECOS_BB)  # Using GUROBI or another MIP-capable solver

# Display results
selected_assets = np.where(binary_inclusion.value > 0.5)[0]  # Assets with inclusion set to 1
optimal_weights = weights.value[selected_assets]
print("Selected Assets:", selected_assets)
print("Optimal Weights:", optimal_weights)

Selected Assets: [ 6 16 18]
Optimal Weights: [0.81293246 0.05       0.13706753]


In [36]:
tt=np.zeros(25)
tt[selected_assets] = optimal_weights
pf_wt=pd.DataFrame(tt)

In [37]:
pf_wt.to_csv(os.path.join(portfolio_wts_dir, 'MVO_new', 'MVO_new_year_n_3_'+str(bt_year)+'.csv'), header=None, index=False)